<a href="https://colab.research.google.com/github/tomasonjo/blogs/blob/master/llm/Marvel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain neo4j openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.4/192.4 kB 22.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.4 MB/s eta 0:00:00
  Created wheel for neo4j: filename=neo4j-5.14.0-py3-none-any.whl size=265476 sha256=2d7084ec539ad04a9e1a102c09c6a66bb01139d0f4aa37b788e01b7456c9c1ad
  Stored in directory: /root/.cache/pip/wheels/53/e0/d7/603097e3fed62f821523433801c09e04cd7a7610c7565bd5a3
Successfully built neo4j
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source o

In [ ]:
from langchain.graphs import Neo4jGraph

url = "bolt://3.231.144.192:7687"
username ="neo4j"
password = "leapers-turbulence-means"
graph = Neo4jGraph(
    url=url,
    username=username,
    password=password
)

In [ ]:
graph.schema

In [ ]:
import os
from getpass import getpass

os.environ["OPENAI_API_KEY"] = getpass('Enter your password: ')

Enter your password: ··········


In [ ]:
from langchain.chains import GraphCypherQAChain
from langchain.chat_models import ChatOpenAI

cypher_chain = GraphCypherQAChain.from_llm(
    cypher_llm = ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo'),
    qa_llm = ChatOpenAI(temperature=0), graph=graph, verbose=True,
    validate_cypher=True
)

In [ ]:
cypher_chain.run(
    "Which 5 heroes have the highest pagerank score?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (h:Hero)
RETURN h.id, h.pagerank
ORDER BY h.pagerank DESC
LIMIT 5
Full Context:
[{'h.id': 'CAPTAIN AMERICA', 'h.pagerank': 4.310106385495555}, {'h.id': 'THOR/DR. DONALD BLAK', 'h.pagerank': 3.3331648191063796}, {'h.id': 'SPIDER-MAN/PETER PARKER', 'h.pagerank': 3.1287218925351423}, {'h.id': 'IRON MAN/TONY STARK', 'h.pagerank': 2.6093660811170616}, {'h.id': 'HULK/DR. ROBERT BRUC', 'h.pagerank': 2.4362290101815867}]

> Finished chain.


'The five heroes with the highest pagerank scores are Captain America, Thor/Dr. Donald Blak, Spider-Man/Peter Parker, Iron Man/Tony Stark, and Hulk/Dr. Robert Bruc.'

In [ ]:
cypher_chain.run(
    "How many comics did CAPTAIN AMERICA appear in?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (:Character {id: 'CAPTAIN AMERICA'})-[:APPEARED_IN]->(comic:Comic)
RETURN COUNT(comic)
Full Context:
[{'COUNT(comic)': 1334}]

> Finished chain.


'CAPTAIN AMERICA appeared in 1334 comics.'

In [ ]:
cypher_chain.run(
    "Which hero appeared in most comics with CAPTAIN AMERICA?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (c:Character)-[:APPEARED_IN]->(:Comic)<-[:APPEARED_IN]-(h:Hero)-[:APPEARED_IN]->(:Comic)<-[:APPEARED_IN]-(ca:Character {id: 'CAPTAIN AMERICA'})
RETURN h.id, COUNT(DISTINCT c) AS numComics
ORDER BY numComics DESC
LIMIT 1


KeyboardInterrupt: ignored

In [ ]:
cypher_chain = GraphCypherQAChain.from_llm(
    cypher_llm = ChatOpenAI(temperature=0, model_name='gpt-4'),
    qa_llm = ChatOpenAI(temperature=0), graph=graph, verbose=True,
    validate_cypher=True
)
cypher_chain.run(
    "Which hero appeared in most comics with CAPTAIN AMERICA?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (c:Character {id: 'CAPTAIN AMERICA'})-[:APPEARED_IN]->(comic:Comic)<-[:APPEARED_IN]-(h:Hero)
RETURN h.id, COUNT(comic) AS NumberOfComics
ORDER BY NumberOfComics DESC
LIMIT 1
Full Context:
[{'h.id': 'IRON MAN/TONY STARK', 'NumberOfComics': 440}]

> Finished chain.


'IRON MAN/TONY STARK appeared in the most comics with CAPTAIN AMERICA, with a total of 440 comics together.'